<h1>Example Langchain with MSSQL Server</h1>

<h4>Initialize the DB</h4>

In [1]:
!pip install pyodbc

     -------------------------------------- 68.7/68.7 kB 943.4 kB/s eta 0:00:00



[notice] A new release of pip available: 22.2.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
from langchain.sql_database import SQLDatabase

# Replace the following with your actual connection details
username = 'user1'
password = 'IoP3X5_QRFiE'
servername = 'localhost'  # or your server's name
databasename = 'AdventureWorks2017'

# MSSQL connection URI
connection_uri = f"mssql+pyodbc://{username}:{password}@{servername}/{databasename}?driver=ODBC+Driver+17+for+SQL+Server"
#connection_uri = f"mssql+pyodbc://{username}:{password}@{servername}/{databasename}?driver=SQL+Server"

db = SQLDatabase.from_uri(connection_uri)
print(db)

In [2]:
from langchain_community.agent_toolkits import (create_sql_agent, SQLDatabaseToolkit)
from langchain_openai import ChatOpenAI
from langchain_community.agent_toolkits.sql.prompt import SQL_FUNCTIONS_SUFFIX
from langchain_core.messages import AIMessage, SystemMessage
from langchain_core.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
)

toolkit = SQLDatabaseToolkit(db=db, llm=ChatOpenAI(temperature=0))
tools = toolkit.get_tools()

messages = [
    HumanMessagePromptTemplate.from_template("{input}"),
    AIMessage(content=SQL_FUNCTIONS_SUFFIX),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
]

prompt = ChatPromptTemplate.from_messages(messages)



In [3]:
from langchain.agents import create_openai_tools_agent
from langchain.agents.agent import AgentExecutor

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

agent = create_openai_tools_agent(llm, tools, prompt)

agent_executor = AgentExecutor(
    agent=agent,
    tools=toolkit.get_tools(),
    verbose=True,
)

In [4]:
agent_executor.invoke({"input": "Describe the schema of the [Sales].[Customer] table"})



> Entering new AgentExecutor chain...

Invoking: `sql_db_list_tables` with ``


AWBuildVersion, DatabaseLog, ErrorLog
Invoking: `sql_db_schema` with `Sales.Customer`


Error: table_names {'Sales.Customer'} not found in databaseI apologize, but it seems that the [Sales].[Customer] table does not exist in the database. Please make sure that the table name is correct and try again.

> Finished chain.


{'input': 'Describe the schema of the [Sales].[Customer] table',
 'output': 'I apologize, but it seems that the [Sales].[Customer] table does not exist in the database. Please make sure that the table name is correct and try again.'}